## Regression Task: Predicting energy of neutrinos 

This week we will use the data from the neutrino experiment, the Oscillation Project with Emulsion-tRacking Apparatus (OPERA). We want to find the relation of the energy of the shower created by a neutrino and the number of hits in the detector.

Let's first load the data and see what are inside. Run the first cell to find the path and names of the datasets.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

As usual there are 'training' and 'test' datasets. For the in-class exercise let's focus on the training data.

Please load the neutrino_train.npz and print out its content in the next cell.

In [ ]:
neutrino_file = np.load('/kaggle/input/phys591000-2022-week03/neutrino_train.npz')

content_names = neutrino_file.files
print(content_names)

As you can see the data contain `Ntotal`, `Nmax`, `izmax` and `Energy`. The meanings are

* `Energy`: Electron neutrino energy

* `Ntotal`: Total number of hits

* `Nmax`: Max hit multiplicity in one layer of the detector

* `izmax`: Max depth of the shower

Make a few plots showing the relations of e.g. Energy v.s. Ntotal, Energy v.s. Nmax. Do the plots make sense?

In [ ]:
Energy = neutrino_file['Energy']
Ntotal = neutrino_file['Ntotal']
Nmax = neutrino_file['Nmax']
izmax = neutrino_file['izmax']

print('Energy shape is:', Energy.shape)
print('Ntotal shape is:', Ntotal.shape)

import matplotlib.pyplot as plt
# Please make some plots showing the relations of the features below
fig = plt.figure(figsize=(6,5), dpi=80)
plt.scatter(Nmax, Energy)

You should see that Energy is almost linearly proportional to the total number of hits (Ntotal). So we can do a linear regression fit to predict Energy from Ntotal!

In the cell below we fit Energy (y-axis) v.s. Ntotal (x-axis) with a line, and plot the fitted line against the input data.

Note that input data for sklearn linear regression need to be two-dimensional arrays of shape (n_samples, n_features). This is the case even if you only have one feature. So we use **np.newaxis** to transform Ntotal to a 2D array.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge

# Build the linear regression model.  
# fit_intercept=False means we think Ntotal = 0 corresponds to Energy = 0
# i.e. no interecept on the y-axis

model = LinearRegression(fit_intercept=False)
x = Ntotal
y = Energy
model.fit(x[:, np.newaxis], y)

# Fitted line from the linear model
xfit = np.linspace(0, np.max(Ntotal), 1000)
yfit = model.predict(xfit[:, np.newaxis])

# compare the fitted line with the data
plt.scatter(x, y)
plt.plot(xfit, yfit, 'r')
plt.ylabel("Energy")
plt.xlabel("Ntotal")
plt.show()

Next we demonstrate how to do a polynomial fit. We need to use **PolynomialFeatures** to 'pre-process' the feature into a polynomail, then use Linear Regression to fit the data with this polynomial. Thus we need to use **Pipeline** to chain these steps together.

In [ ]:
# Polynomial Regression test
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

# include_bias=False to set the intercept on the y-axis to 0 for polynomial fit
# For fun try a 5th degree polynomial
model_poly = Pipeline([('poly', PolynomialFeatures(degree=5, include_bias=False)),
                       ('linear', LinearRegression(fit_intercept=False))])

result_poly = model_poly.fit(x[:, np.newaxis], y)

# Get the output coefficients in the order of the result 'get_feature_names()'
model_poly.steps[0][1].get_feature_names()
coef = result_poly['linear'].coef_
print("Coefficients from polynomial regression are: ", coef)

What does the fitted polynomial look like? Please plot the fitted polynomial and compare to the input data in the cell below.

In [ ]:
# Plot the fit result from the polynomial fit and the input data on the same plot

# Plot using model.predict
xfit = np.linspace(0, np.max(Ntotal), 100)
yfit = model_poly.predict(xfit[:, np.newaxis])

plt.scatter(x, y)
plt.plot(xfit, yfit, 'r')
plt.ylabel("Energy")
plt.xlabel("Ntotal")
plt.show()

In the next cell we'll demonstrate the effect of overfitting and how to regularize it. As a toy model, we will use only 4 events (samples) from the data, and we will use the feature 'Nmax'. First we'll fit it with a 4th degree polynomial.

In [ ]:
# Select 4 events from the input

wanted_point = np.arange(0,120, 30)
x_1= Nmax[wanted_point]
y_1 = Energy[wanted_point]

overfit = Pipeline([('poly', PolynomialFeatures(degree=4)),
                       ('linear', LinearRegression())])

result_overfit = overfit.fit(x_1[:, np.newaxis], y_1)

# Plot using model.predict
x_1_fit = np.linspace(np.min(x_1), np.max(x_1), 100)
y_1_fit = overfit.predict(x_1_fit[:, np.newaxis])

plt.scatter(x_1, y_1)
plt.plot(x_1_fit, y_1_fit,'r')
plt.show()

As you can see we make a 'perfect' fit of the 4 points. But can this perfect fit generalize to other data points? Let's try out in the next cell.

In [ ]:
more_points = np.arange(0,120, 10)
x_2= Nmax[more_points]
y_2 = Energy[more_points]

# make a plot comparing the fitted result from the previous cell to this set of data
x_2_fit = np.linspace(np.min(x_2), np.max(x_2), 100)
y_2_fit = overfit.predict(x_2_fit[:, np.newaxis])

plt.scatter(x_2, y_2)
plt.plot(x_2_fit, y_2_fit,'r')
plt.show()

How about we fit the 4 points with some regularization? In the next cell we will show how to make a polynomial fit with Lasso regularization.

In [ ]:
# Try regularization
# alpha is the coefficient of the penalty term

model_Lasso = Pipeline([('poly', PolynomialFeatures(degree=4)),
                ('linear', Lasso(alpha=1))])

result_Lasso = model_Lasso.fit(x_1[:, np.newaxis], y_1)

# Please compare the fitted result with the 4 points *and* the set from 'more_points'

# Plot using model.predict
x_3_fit = np.linspace(np.min(x_1), np.max(x_1), 100)
y_3_fit = result_Lasso.predict(x_3_fit[:, np.newaxis])

plt.scatter(x_1, y_1)
plt.plot(x_3_fit, y_3_fit,'r')
plt.show()

x_4_fit = np.linspace(np.min(x_2), np.max(x_2), 100)
y_4_fit = result_Lasso.predict(x_4_fit[:, np.newaxis])

plt.scatter(x_2, y_2)
plt.plot(x_4_fit, y_4_fit,'r')
plt.show()